# 時系列分析

## 本メモ（時系列分析と状態空間モデルの基礎　RとStanで学ぶ理論と実装）

## 必要知識
- データ生成過程（DGP）
    - 今回取得できたデータは、パラレルワールドの中の1回で取得できたデータだと考える
    - →　母集団は、パラレルワールドの全体
    - データ生成過程とは、取得できたデータがどのような確率分布から取得できたのかを考えること
- 時系列分析の目的：時系列モデルを推定すること
- ホワイトノイズ
    - 未来予測の情報がない雑音
    - 期待値が０で、分散が一定、自己相関が０
- ランダムウォーク
    - iid系列の累積和
- 構造
    - 特徴はデータの前後に関係があること
    - 自己相関をグラフにする　→　コレログラム
    - 時系列データ　＝　短期の自己相関
           ＋周期変動
           ＋トレンド
           ＋外因性
           ＋ホワイトノイズ
- 相関
    - 自己相関：k時点前との間にある相関
    - 偏自己相関：k-1時点との相関をなくしたk時点との相関
                - 「1時点まででは表現できなかった残り」同士で相関をとる
    - iid系列
        - データが独立である条件
        - ホワイトノイズとの違い
            - iidは平均０とは限らない
            - ホワイトノイズは独立のみ。iidは独立かつ同一分布
            - iidは強定常、ホワイトノイズは弱定常
            - 錯乱項を発生させるには、 ホワイトノイズで十分
            - ランダムウォークのみiidを仮定（ARやMAはホワイトノイズを仮定）
        - iid系列　　　　　→　強定常：常に同じ確率分布
        - ホワイトノイズ　→　弱定常：時間差のみに依存
    - 定常性
        - 平均と自己共分散が一定であること
    - そもそも時系列は分析しずらいので、「分析しやすいデータ」のみを扱う　→　定常性
        - 定常過程　→　平均が一定　
            - 基本的なデータはトレンドがあったり、季節性があったりと、ほぼ定常ではない
        - でも定常仮定でないと分析できないので、差分をとったりして、差分系列を作る
            - 原系列　→　非定常過程　かつ　差分系列　→　定常過程　の場合を単位根過程（1次和分過程）
        - ランダムウォークはホワイトノイズの累積和なので非定常だが、差分を取れば定常になる
            
****
            
##  分析手法
 - Box-Jenkins法の分析手順<br>
    Step1　：　データを分析やすいように変換<br>
    Step2　：　データをモデル化<br>
    Step3　：　モデルの評価<br>
    Step4　：　モデルで予測する<br>

    - ARモデル
        - 前地点のデータと相関がある場合
        - yt = c + φ1yt-1 + εt　c：定数　φ：係数
        - φ > 1：過去データの影響がずっと残る
        - φ = 1 ：ランダムウォーク
        - φ  < 1：過去のデータの影響がほぼなくなる
        -　常に定常ではない（|φ|<1のみ定常）
    - MAモデル
        - 自己相関がある場合
        - 常に定常
        - 反転可能性によって、「良い性質」かを判断可能
            - 反転可能性とは、「予測誤差（ε)の大小を、過去のデータから判断できるかどうか」
            - 反転可能性の条件　→ 特性方程式において、解の絶対値が1より大きいこと
            - 上記を満たすと、予測誤差（ε)の大小を、過去のデータから判断できる
            - ※εを過去データのΣで表現する事ができる為
    - ARとMAの違い
        - AR：過去のある時点における自分のデータ、偏自己相関に対応
        - MA：過去の複数地点の移動平均における自分のデータ、自己相関に対応
    - ARIMA
        - 和分過程の場合、ARMAは使えないので、ARIMAにして差分系列にしてから使う
    - SARIMA
        - ARIMAに季節性を取り入れたモデル
        - SARIMAでは、ARIMAで使った「1時点前との変化」＋「1季節前との変化」によってモデル化
    - ARIMAX
        - 「回帰」要素を取り入れたARIMA
        - ダミー変数を使って、曜日や祝日の効果を組み込める
- 分析モデルの次数の決め方
    - ３つの作業
        - 次数の決定：モデル選択
        - 差分の決定：単位根検定
        - モデルの評価
    - 次数の決定：AICを使って、手当たり次第に評価してみる
        - AICを使う理由：過学習を抑えるため
    - 差分の決定：KPSS検定かADF検定
        - KPSS
            - 帰無仮説：単位根なし
            - 対立仮設：単位根あり
        - ADF
            - 帰無仮説：単位根あり
            - 対立仮設：単位根なし
        - 使い分け
            - ADF：検定回帰式に、「被説明変数のラグ項」を含める
    - 定常性と反転可能性の確認（ARMAの場合）
        - 定常性
            - MA項は常に定常なため、AR項の定常性かどうかがARMAの定常性となる
            - AR項は定常なら常に反転可能なため、MA項の反転可能かどうかがARMAの反転可能な条件となる
    - ARIMAの次数決定アルゴリズム<br>
        1 : KPSS検定にて、単位根の有無を調べる<br>
        2 : 差分系列に対して、ARIMAモデルのp,q次数を手当たり次第作成<br>
        3 : 定常性・反転可能性をチェック<br>
        ４ : 定常性・反転可能性チェックを通った中でAICが最も低いモデルを選ぶ<br>
    - モデル評価
        - 残差がホワイトノイズである　→　うまくモデル化できている
        - 残差がホワイトノイズでない　→　まだモデルに反映できる要素が残っている
        - 残差に自己相関があるかを検定（ホワイトノイズかどうか）
            - リュング・ボックス検定
                - 帰無仮説：k時点までのデータが全て自己相関0
                - 対立仮設：いずれかが０ではない
        - 当てはめ残差が正規分布かを検定（残差は正規分布を仮定したホワイトノイズのため）
            - ジャック・ベラ検定
                - 帰無仮説：正規分布
                - 対立仮設：正規分布でない
***

## 時系列分析で気をつける点
- めも：最小二乗法では時系列データがうまくいかない理由
    - データが独立ではない場合が多い　→　昨日の売上が今日の売上に関係している場合が多い
- みせかけの回帰
    - 原因は、残差に自己相関があること
        - 最小二乗法が有効でなくなる
    - 最小二乗法（OLS）ではなく、一般最小二乗法（GLS）を使う → Prais–Winsten法
        - AR(1)とすると、yt = β1 + β2xt + εt
        - 残差εから自己相関を想定した回帰式を作り、その係数からytの推定値を作る
        - ytの推定値は残差の自己相関を取り除いているので、時系列モデルを組む
    - ダービン・ワトソン検定
        - 線形回帰の結果に対して、自己相関の有無を調べる
        - 帰無仮説：自己相関０
        - 対立仮設：０ではない
    - 防ぐ方法
        - 過去のデータをモデルに組み込み、データの持つ自己相関を表現するモデルを作成（ARIMAXやVAR）
        - 差分系列の回帰分析をとる
            - 共和分
                - 以下を満たす
                    - 2つのデータどちらも単位根過程
                    - 2つのデータは線形結合すると単位根過程でなくなる
- VAR
    - 多変量の時系列モデル
        - お互いに影響を及ぼす時に考える　※どちらか一方的な場合はARIMAXを使う
        - 例：収入と消費　→　収入が増えれば消費が増える事が仮定できる
        - yt = c + φ1yt-1 + ... + φpyt-p + εt （t:n次ベクトル、c:n×1次定数ベクトル、φ:n×n次元係数行列）
    - グレンジャー因果検定
        - 相手のデータを使う事で、予測精度が上がるかどうか　→ 直接の因果ではなく、ほぼ相関に近いよう
        - 今年の収入　＝　φ11去年の収入　＋　φ12去年の消費　＋　c + ε<br>
          今年の収入　＝　φ21去年の収入　　　　　　　　　　　＋ c +　ε
           

-----

#### ARCH / GARCH
    - 目的：時間によって、分散が変動するデータを表現するモデル
    - 基本は、ファイナンスデータで使われる。
    - 分散が分かれば、どれだけブレ幅があるのかが分かるので、最大でどの程度の損をするのかが分かる

### 4部　状態空間モデル
    特徴
    - モデルの自由度が高い
    - 自動化が困難
    - Box-Jenkins法の比べて、属人性が高い

#### 1章　状態空間モデルとは？
    メリット
    - 知見や感覚をモデル化可能
    - モデルの解釈が簡単
    - 差分をとる等の前処理が不要
    - 欠損値があってもそのまま分析できる
    デメリット
    - 分析ルールが決まっていない

「モデルのデータ表現」と「パラメタ推定」を分けて考えることが重要<br>
→　「パラメタ推定」はパッケージで最悪なんとかなるが、「モデルのデータ表現」は必ず自分で組む必要がある


モデルのデータ表現<br>
「状態方程式」と「観測方程式」で表される<br>
状態方程式：状態の変化のプロセスを記述　→　前時点の状態を用いた予測値＋過程誤差<br>
観測方程式：状態から観測値が得られるプロセスを記述　→　状態　＋　観測誤差

パラメタ推定<br>
最も有名なパラメタ推定は、カルマンフィルタ＋最尤法<br>
カルマンフィルタ：目に見えない「状態」を効率よく推定<br>
最尤法：パラメタの推定
→　状態とパラメタの推定に分かれている

***

### 5部　状態空間モデルとカルマンフィルタ
状態方程式： x t = T t * x t-1 + R δt　（δ ~ N(0,Qt)）<br>
観測方程式：y t = Z t * x t + ε　（ε ~ N(0,Ht)）

#### 状態方程式と観測方程式における推定の流れ
1. 状態方程式から、過去の状態を用いて未来の状態を予測　→　未来の観測を予測
2. 実際の観測値から、予測した未来の状態を補正
3. 未来の状態から、再度未来の観測値を予測

#### 平滑化
全てのデータが手に入ったあとで、状態の補正を行う
過去の状態を、過去の観測値と未来の観測値を使って補正する

#### 最尤法
状態の過程誤差と観測の観測誤差というパラメタを最尤法で推定

#### 線形ガウス状態空間モデル（カルマンフィルタで作った状態空間モデル）の推定の流れ
※誤差が正規分布に従っていることを仮定したモデル<br>
1. 状態方程式・観測方程式を用いてモデルを表現
2. 「とりあえず」のパラメタを使ってフィルタリング（状態の補正）
3. カルマンフィルタの結果を援用し、最尤法を用いてパラメタを推定
4. 推定されたパラメタを用いて、再度フィルタリング
5. 推定されたパラメタを用いて、平滑化

#### 2章　状態方程式・観測方程式による表現技法
状態空間モデルは、トレンドや季節性を表現できる

線形回帰を状態方程式・観測方程式すると？<br>
線形回帰（切片のみの場合）： yt = α + vt　vt ~ N(0,σ)<br>
↓　書き換えると<br>
状態方程式：xt = α<br>
観測方程式：yt = xt + vt vt ~ N(0,σ)<br>
つまり、状態は一定で、観測誤差の分散だけがある

#### ローカルレベルモデル
過程誤差と観測誤差を両方認めた比較的簡単なモデル<br>
状態方程式：xt = xt-1 + εt　ε ~ N(0,σ）<br>
観測方程式：yt = xt + δt  δ ~N(0,σ）<br>
状態方程式から、ホワイトノイズの累積和＝ランダムウォーク<br>
観測方程式は、ランダムウォークに観測誤差を加えたデータ<br>
→　状態空間モデルで、非定常過程（ランダムウォーク）を表現できた！！！<br>
→　ただし、過程誤差と観測誤差の期待値が０のため、ローカルレベルモデルの期待値が一定になる。<br>
　　→　役立たない・・・www

#### ローカル線形トレンドモデル
ローカルレベルモデルに、トレンド成分を追加したモデル<br>
トレンド：δt = δt-1 + 残差　残差 ~ N(0,σ）<br>
状態方程式：xt = xt-1 + δt-1 + 残差　残差 ~ N(0,σ2)<br>
観測方程式：yt = xt + 残差　残差 ~ N(0,σ3)<br>

多次元への拡張は、いつも通り行列に置き換えることで可能<br>
線形ガウス状態空間モデルを行列表現すれば、ローカルレベルモデルもローカル線形トレンドモデルも表現可能


#### 周期的変動モデル
季節性の周期変動をモデル化するのは2通り<br>
- ダミー変数を使う
- 三角関数を使う

ダミー変数を使う<br>
x1,t = -x1,t-1 -x2,t-1 -x3,t-1 + 残差　残差 ~ N(0,σ)<br>
x2,t = x1,t-1<br>
x3,t = x2,t-1<br>
yt = x1,t + 残差　残差 ~ N(0,σ1）<br>

#### 基本構造時系列モデル
時系列データ　＝　トレンド　＋　周期変動　＋　ホワイトノイズ<br>
といった、モデルを状態空間モデルで表すと、<br>
yt = xt + γt + ε　ε ~ N(0,σ）<br>
となり、差分をとって・・・といったことを気にしなくてもよくなる<br>
ARIMAXと同じように、外生変数を組み込むことも可能<br>

#### 3章　カルマンフィルタ
カルマンフィルタ：「状態の１期先への予測」と「観測値を用いた状態の補正」を繰り返して、パラメータを推定する方法<br>
「観測値を用いた状態の補正」は以下でおこなう<br>
補正後の状態　＝　補正前の状態　＋　カルマンゲイン（実際の観測値　ー　予測の観測値）<br>
カルマンゲインは常に１以下<br>

カルマンゲイン　＝　状態の予測誤差の分散　/　（状態の予測誤差の分散＋観測誤差の分散）<br>

#### 4章　散漫カルマンフィルタ
カルマンフィルタの弱点　→　初期値パラメータに結果が依存する<br>
そこで、散漫初期化という手法をとる<br>
アイデア<br>
- 状態の初期値は分からないので諦める
- その代わり、「状態の予測誤差の分散」の初期値を∞にする

#### 5章　平滑化（スムージング）
考え方<br>
今日のデータを使って、今日のデータを補正　→　フィルタリング<br>
今日のデータを使って、昨日より前のデータを補正　→　平滑化<br>

移動平均をとったような、滑らかなデータになる

---

### 6部　状態空間モデルとベイズ推論
非線形かつ非正規分布の状態空間モデルを設定する

結局、GLMMのMCMCを使ったパラメータ推定と同じく、<br>
パラメータを積分して推定することが難しいのでモンテカルロ法を使って推定を行う

#### 1章　一般化状態空間モデルとベイズ推論
データの傾向が「想像」できるときに使うモデル<br>
ex.生物数の上限があれば、データの上限もあることが予測される。

モデルが複雑になりがちだから、最尤法ではなく、MCMC（HMC法）とベイズ推論を使用して、パラメータを推定する<br>


## 不明点
### 21/12/29時点
 - 状態空間モデルとは？
 - カルマンフィルタとは？
 - 錯乱項ならホワイトノイズでも良いのに、ランダムウォークのみiidを仮定する意味は？
 - 差分を決める際に、KPSSとADFの両検定における違いは？
 - 時系列モデルで、実装している例ってある？

***

## 関係性
### 21/12/29時点
 - ARとVRの違い<br>
     - AR：過去の自分と関係性がある
     - MA：自己相関がある